# Analysis goals

1. Determine what data is relevant to show by a "results" object and its corresponding stats/plots.
2. Identify bugs or conceptual errors.
3. Examine runtime bottlenecks and determine how to speed up sim.

In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import pstats
import numpy as np

pd.set_option("display.max_columns", None)

In [2]:
with open("../pickles/scenario.pkl", "rb") as f:
    scenario = pickle.load(f)
with open("../pickles/metrics_processor.pkl", "rb") as f:
    metrics_processor = pickle.load(f)

In [8]:
market = list(scenario.markets.values())[0]
print(market)
reg = market.models[0][1]
size = int(1e6)
x = np.array([size]).reshape(-1, 1)

External Market (USDC, USDT)


In [81]:
def func(x):
    T = np.clip(x, reg.X_min_, reg.X_max_)
    return reg.f_(T)

In [57]:
%timeit reg.predict(x)

96.6 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [61]:
%timeit func(x)

17.4 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [72]:
func(x) == reg.predict(x)

array([[ True]])

In [82]:
func(x).dtype

float64


dtype('float64')

In [79]:
reg.predict(x).dtype

dtype('float64')

In [80]:
x.dtype

dtype('int64')

In [76]:
from sklearn.utils import check_array

T = check_array(x, dtype=np.float64, ensure_2d=False)
T.dtype

dtype('float64')

In [44]:
reg._check_input_data_shape(T)

In [46]:
T = np.clip(T, reg.X_min_, reg.X_max_)
T

array([[5.04572974e+08]])

In [40]:
reg.predict(x)

array([0.0002195])

In [41]:
reg.f_(x)

array([[nan]])

In [12]:
df = metrics_processor.df
df

,timestamp,arbitrageur_profit,arbitrageur_count,keeper_profit,keeper_count,liquidator_profit,liquidator_count,llamma_WETH_price,llamma_WETH_oracle_price,llamma_WETH_fees_x,llamma_WETH_fees_y,llamma_WETH_bal_x,llamma_WETH_bal_y,controller_WETH_system_health,controller_WETH_bad_debt,controller_WETH_num_loans,controller_WETH_debt,stableswap_crvUSD_TUSD_price,stableswap_crvUSD_TUSD_ma_price,stableswap_crvUSD_TUSD_lp_supply,stableswap_crvUSD_TUSD_virtual_price,stableswap_crvUSD_TUSD_TUSD_bal,stableswap_crvUSD_TUSD_crvUSD_bal,stableswap_crvUSD_USDP_price,stableswap_crvUSD_USDP_ma_price,stableswap_crvUSD_USDP_lp_supply,stableswap_crvUSD_USDP_virtual_price,stableswap_crvUSD_USDP_USDP_bal,stableswap_crvUSD_USDP_crvUSD_bal,stableswap_crvUSD_USDC_price,stableswap_crvUSD_USDC_ma_price,stableswap_crvUSD_USDC_lp_supply,stableswap_crvUSD_USDC_virtual_price,stableswap_crvUSD_USDC_USDC_bal,stableswap_crvUSD_USDC_crvUSD_bal,stableswap_crvUSD_USDT_price,stableswap_crvUSD_USDT_ma_price,stableswap_crvUSD_USDT_lp_supply,stableswap_crvUSD_USDT_virtual_price,stableswap_crvUSD_USDT_USDT_bal,stableswap_crvUSD_USDT_crvUSD_bal,pk_crvUSD/TUSD_debt,pk_crvUSD/TUSD_profit,pk_crvUSD/USDP_debt,pk_crvUSD/USDP_profit,pk_crvUSD/USDC_debt,pk_crvUSD/USDC_profit,pk_crvUSD/USDT_debt,pk_crvUSD/USDT_profit,aggregator_price,stablecoin_total_supply
0,2023-12-06 22:47:19,16031.435765,4,0.0,0,65012.788847,1,2245711730308338024482,2252689216320075464704,2636248,0,794465331942950411210999,12646117928609448025432,4.061528e+17,6582755539915736668938240,243,19766309101639759549598668,997687123610722464,997178462699128223,4602598847318464060909400,1000001125546496117,1368312143212577915082621,3235195647056719331952849,995557164758900273,994925052636487860,4983836723625910875895581,1000000663317038117,1126911145615430252820820,3859061453197663631989558,995989820777331057,994080592987261800,34745774104237219907970604,1000002113123312899,8222838334836,26536323616655163012812937,995325577283750764,994136837577183132,31453878236842814267581273,1000001305488233112,6951259199869,24516855683989434422548571,0,0,0,0,0,0,0,0,994151975553486414,321506617398337923273370681
1,2023-12-06 23:47:19,16327.928812,7,0.0,0,65013.931350,2,2244934322013356681333,2248746093350448456143,3979890,0,883525463828914581431284,12606572595980065876253,4.037375e+17,6582755539915736668938240,242,19766528976881117324695679,997044713325545490,997156462847939844,4602598847318464060909400,1000002510287713818,1241082541342113844243127,3362765821144537625206245,996516552402313498,996363077226241005,4983836723625910875895581,1000001941706078675,1253991554890279978072404,3731484899963226285916218,996086475129871192,995959968085183934,34745774104237219907970604,1000002241172679910,8311585666835,26447227759450408232963984,995325577283750764,995306965277382070,31453878236842814267581273,1000001305488233112,6951259199869,24516855683989434422548571,0,0,0,0,0,0,0,0,995737594659636573,321723278974615328754799266
2,2023-12-07 00:47:19,17120.862197,9,0.0,0,65013.931350,2,2244912297744144078326,2248762332647248126239,4952489,0,883525463828914581431284,12606572595980065876253,4.037188e+17,6582755539915736668938240,242,19766840537701053496330243,992688136607587679,993767959369346049,4602598847318464060909400,1000007037344698179,826293923707326188520087,3779539900798158850032856,996887967173983255,996879755190529775,4983836723625910875895581,1000002558557547977,1315342828453460779006292,3669933970896803736718297,996444204304521228,996436628332759163,34745774104237219907970604,1000002751777424286,8665521894769,26091983768075148070125332,995325577283750764,995325282180247631,31453878236842814267581273,1000001305488233112,6951259199869,24516855683989434422548571,0,0,0,0,0,0,0,0,995915857404862901,322140053054268949979625877
3,2023-12-07 01:47:19,17972.936330,13,0.0,0,65013.931350,2,2236181810574026409535,2240157459709222562666,6645836,0,1020838271090489141296674,12545303998771444257372,3.984447e+17,6582755539915736668938240,242,1976715

In [13]:
# # LLAMMA price
# (df["llamma_WETH_price"].astype(float) / 1e18).plot("Spot Price")
# (df["llamma_WETH_oracle_price"].astype(float) / 1e18).plot(label="Oracle Price")
# # TODO does it make sense that oracle price is a muted version of LLAMMA price?

## Runtime Profile

In [16]:
prof = pstats.Stats("../logs/sim.prof")
prof.sort_stats(pstats.SortKey.CUMULATIVE)
prof.print_stats()

Wed Dec  6 22:54:23 2023    ../logs/sim.prof

         294239805 function calls (287066865 primitive calls) in 424.300 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002  424.360  424.360 /root/crvUSDrisk/src/sim/__init__.py:16(sim)
      168    0.003    0.000  400.850    2.386 /root/crvUSDrisk/src/sim/scenario.py:166(perform_actions)
      168    0.004    0.000  374.932    2.232 /root/crvUSDrisk/src/agents/arbitrageur.py:35(arbitrage)
      475    0.021    0.000  374.490    0.788 /root/crvUSDrisk/src/agents/arbitrageur.py:92(find_best_arbitrage)
     9500    0.038    0.000  374.469    0.039 /root/crvUSDrisk/src/trades/cycle.py:77(optimize)
     7658    0.073    0.000  373.410    0.049 /root/crvUSDrisk/src/trades/cycle.py:182(_optimize_mem)
     7658    0.132    0.000  368.872    0.048 /root/crvUSDrisk/venv/lib/python3.11/site-packages/scipy/optimize/_minimize.py:754(minimize_scalar)
     7658   1